# Aim
Repeat earlier analyses like in step4 but:
* without ARP clade
* without spuriouos CDC5 sequence

+ Add arabidopsis mybs to annotate their subfamily as in J&R
+ Annotate the tree with RNA seq data for Azolla filiculoides
+ Add more _Azolla filiculoides_ MYB sequences

All sequences are linear already, so I can start composing the fasta file and aligning right away.

In [ ]:
cat data/CDC5-outgroup_sequences_linear.fasta \
    data/I_sequences_linear.fasta    \
    data/II_sequences_linear.fasta   \
    data/III_sequences_linear.fasta  \
    data/IV_sequences_linear.fasta   \
    data/V_sequences_linear.fasta    \
    data/VI_sequences_linear.fasta   \
    data/VII_sequences_linear.fasta  \
    data/VIII_sequences_linear.fasta \
    data/Azfi-mybs-subfamVI-suspects_linear.fasta \
    data/R1R2R3_sequences_linear.fasta \
    data/arabidopsis-myb_sequences.fasta \
    data/Azfi-v1-MYB-sequences.fasta \
    > data/combi-I-to-VIII-Azfi-Arabidopsis_sequences_linear.fasta
    

In [ ]:
inseq=combi-I-to-VIII-Azfi-Arabidopsis_sequences_linear

In [ ]:
rm ./data/alignments_raw/"$inseq"_aligned-mafft-einsi.*

In [ ]:
conda activate phylogenetics
if    [ ! -d ./data/alignments_raw/ ]
then  mkdir  ./data/alignments_raw
fi
for   i in data/combi*sequences_linear.fasta
do    if    [ ! -f "./data/alignments_raw/$inseq"_aligned-mafft-einsi.fasta ]
      then  echo "aligning $inseq"
            einsi --thread $(nproc) data/$inseq.fasta > ./data/alignments_raw/"$inseq"_aligned-mafft-einsi.fasta \
                                                      2> ./data/alignments_raw/"$inseq"_aligned-mafft-einsi.log
      fi
done
conda deactivate

In [ ]:
conda activate jalview
for   i in data/alignments_raw/*.fasta
do    prefix=$(echo $i | sed 's/\.fasta//')
      if    [ ! -f $prefix.png ]
      then  jalview -nodisplay \
                    -open $prefix.fasta \
                    -colour CLUSTAL \
                    -png  $prefix.png > /dev/null 2> /dev/null
      fi
done
conda deactivate

The linsi alignment looks like this:
![](data/alignments_raw/combi-I-to-VIII-Azfi-Arabidopsis_sequences_linear_aligned-mafft-einsi.png)

That looks quite a bit more sparse than the einsi alignments I have seen before, likely as a consequence of adding all these recent arabidopsis sequences.

In [ ]:
ls  data/alignments_trimmed/"$inseq"*

In [ ]:
conda activate phylogenetics
if    [ ! -d data/alignments_trimmed ]
then  mkdir  data/alignments_trimmed 
fi

gt=( 4 5 6 )

# define appendix only once here:
for g in ${gt[@]}
do  trimappendix=trim-gt"$g"
    for a in "data/alignments_raw/$inseq"_aligned*.fasta
    do  appendix=$(echo $a | cut -d '/' -f 3- | sed "s/$inseq\_//" | sed "s/.fasta//")
        if    [ ! -f "$appendix"_"$trimappendix".fasta ]
        then  echo "trimming alignment $a with gap threshold $g"
              sed -i 's/ /_/g' $a
              trimal -in $a   \
                     -out data/alignments_trimmed/"$inseq"_"$appendix"_"$trimappendix".fasta \
                     -gt ."$g" \
                     -htmlout data/alignments_trimmed/"$inseq"_"$appendix"_"$trimappendix".html &
        fi
    done
done
wait
conda deactivate

In [ ]:
conda activate jalview
for   i in data/alignments_trimmed/*.fasta
do    prefix=$(echo $i | sed 's/\.fasta//')
      if    [ ! -f $prefix.png ]
      then  jalview -nodisplay \
                    -open $prefix.fasta \
                    -colour CLUSTAL \
                    -png  $prefix.png > /dev/null 2> /dev/null &
      fi
done
wait
conda deactivate

### 60%
![einsi 6 trimmed](data/alignments_trimmed/combi-I-to-VIII-Azfi-Arabidopsis_sequences_linear_aligned-mafft-einsi_trim-gt6.png)

### 50%
![50](data/alignments_trimmed/combi-I-to-VIII-Azfi-Arabidopsis_sequences_linear_aligned-mafft-einsi_trim-gt5.png)

### 40%
![](data/alignments_trimmed/combi-I-to-VIII-Azfi-Arabidopsis_sequences_linear_aligned-mafft-einsi_trim-gt4.png)

~~Based on past experience, I'm inclined to got with the 50% collumn content alignment, it looks most like the alignments I have used in step 4.~~

I'm having strong doubts about the last two sequences in the MSA, I'll proceed for now, but likely reitterate over this notebook again to take them out.

In [49]:
conda activate phylogenetics
for a in data/alignments_trimmed/"$inseq"_aligned*gt*.fasta
do  #iqpendix='iqtree-b100'
    iqpendix='iqtree-bb2000-alrt2000'

    file_appendix=$(echo $a | cut -d '/' -f 3- | sed "s/$inseq\_//" | sed "s/.fasta//")
    
    if   [ ! -d    analyses/"$inseq"_trees/"$file_appendix" ]
    then echo "Making a directory $file_appendix to store trees (name based on alignment filename)"
         mkdir -p  analyses/"$inseq"_trees/"$file_appendix" 
    fi

    iqprefix=analyses/"$inseq"_trees/"$file_appendix"/"$inseq"_"$file_appendix"_"$iqpendix"
    if   [ ! -f "$iqprefix".treefile ]
    then echo "making a tree of file $a"
         nice iqtree -s $a \
                     -m MFP \
                     -bb 2000 -alrt 2000 \
                     -nt AUTO \
                     -ntmax $(nproc)  \
                     -pre  "$iqprefix" \
                     2>   "$iqprefix".stderr \
                      >    "$iqprefix".stdout
    #cat "$iqprefix".log | mail -s "IQtree_run $a" laura
    fi
done
conda deactivate

(phylogenetics) Making a directory aligned-mafft-einsi_trim-gt4 to store trees (name based on alignment filename)
making a tree of file data/alignments_trimmed/combi-I-to-VIII-Azfi-Arabidopsis_sequences_linear_aligned-mafft-einsi_trim-gt4.fasta
Making a directory aligned-mafft-einsi_trim-gt5 to store trees (name based on alignment filename)
making a tree of file data/alignments_trimmed/combi-I-to-VIII-Azfi-Arabidopsis_sequences_linear_aligned-mafft-einsi_trim-gt5.fasta
Making a directory aligned-mafft-einsi_trim-gt6 to store trees (name based on alignment filename)
making a tree of file data/alignments_trimmed/combi-I-to-VIII-Azfi-Arabidopsis_sequences_linear_aligned-mafft-einsi_trim-gt6.fasta
(phylogenetics) 

I accidentally ran trees of all three trimmings, so let's have a look anyway.

In [ ]:
cd ./analyses/"$inseq"_trees/
grep 'Best-fit model' */*.log
cd ../../

In [ ]:
conda activate phylogenetics
for a in data/alignments_trimmed/"$inseq"_aligned*gt*.fasta
do  iqpendix='iqtree-b200'
    #iqpendix='iqtree-bb2000-alrt2000'

    file_appendix=$(echo $a | cut -d '/' -f 3- | sed "s/$inseq\_//" | sed "s/.fasta//")
    
    if   [ ! -d    analyses/"$inseq"_trees/"$file_appendix" ]
    then echo "Making a directory $file_appendix to store trees (name based on alignment filename)"
         mkdir -p  analyses/"$inseq"_trees/"$file_appendix" 
    fi

    iqprefix=analyses/"$inseq"_trees/"$file_appendix"/"$inseq"_"$file_appendix"_"$iqpendix"
    if   [ ! -f "$iqprefix".treefile ]
    then echo "making a tree of file $a"
         nice iqtree -s $a  \
                     -m MFP \
                     -b 200 \
                     -nt AUTO \
                     -ntmax $(nproc)  \
                     -pre  "$iqprefix" \
                     2>   "$iqprefix".stderr \
                      >    "$iqprefix".stdout
    #cat "$iqprefix".log | mail -s "IQtree_run $a" laura
    fi
done
conda deactivate

(phylogenetics) making a tree of file data/alignments_trimmed/combi-I-to-VIII-Azfi-Arabidopsis_sequences_linear_aligned-mafft-einsi_trim-gt4.fasta


### Intermediate conclusion
In hindsight, I would like to have more _Azolla_ sequences in this analysis. 
At the very least the top 10 RNA seq hits, but perhaps all Azolla mybs.